# Kuett & Catena et al. (Nature Cancer, 2021)

In [1]:
#!python -m pip install deepcell "napari[all]" numpy pandas tifffile "typing-extensions>=3.10.0.0"

In [2]:
from pathlib import Path

import napari
import numpy as np
import pandas as pd
import tifffile
from deepcell.applications import Mesmer

2022-03-04 16:14:07.739994: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jonas/miniconda3/envs/deepcell/lib/python3.9/site-packages/cv2/../../lib64:
2022-03-04 16:14:07.740011: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
raw_img_dir = "raw/MainHer2BreastCancerModel/main_model_201710_unaligned_tiff/201710"
raw_panel_file = "raw/MainHer2BreastCancerModel/model201710_panel.csv"
raw_channel_order_file = "raw/MainHer2BreastCancerModel/channel_order_model201710.csv"

layer_pairs = [
    ("28_9_ROI 10_10.tiff", "27_8_ROI 09_9.tiff"),
    ("54_1_ROI 01_1.tiff", "55_2_ROI 02_2.tiff"),
    ("79_6_ROI 06_6.tiff", "80_7_ROI 07_7.tiff"),
    ("106_12_ROI 13_13.tiff", "107_13_ROI 14_14.tiff"),
    ("129_2_ROI 02_2.tiff", "130_3_ROI 03_3.tiff"),
]

shared_channels = [
    "Histone H3",
    "CD68",
    "CD45",
    "vWF+ CD31",
    "panCK",
    "SMA",
    # "MPO",
    # "Ir191",
    # "Ir193",
]

source_channels = shared_channels + [
    "CK7",
    "CK14",
    "E/P-Cadherin",
    "CD44",
    "pS6",
    "Collagen I",
    "CD3",
    "CD8a",
    "cPARP+cCasp3",
]

target_channels = shared_channels + [
    "CK19",
    "CK5",
    "HER2 (bis)",
    "CK8/18",
    "phospho-H3",
    "Vimentin",
    "CD20",
    "CD138",
    "Ki-67",
]

shared_nuclear_channels = ["Histone H3"]

## Data download

1. Download the file `MainHer2BreastCancerModel.zip` from https://zenodo.org/record/5782846 (version 3)
2. Extract its contents of `MainHer2BreastCancerModel.zip` to the `raw` directory (create the directory if it does not exist)
3. Extract the contents of `raw/MainHer2BreastCancerModel/main_model_201710_unaligned_tiff.zip` to the `raw/MainHer2BreastCancerModel` directory.

## Panel creation

In [5]:
raw_panel = pd.read_csv(raw_panel_file)

source_panel = raw_panel[raw_panel["clean_target"].isin(source_channels)]
source_panel = source_panel.rename(columns={"clean_target": "name"})
source_panel.to_csv("source_panel.csv", index=False)

target_panel = raw_panel[raw_panel["clean_target"].isin(target_channels)]
target_panel = target_panel.rename(columns={"clean_target": "name"})
target_panel.to_csv("target_panel.csv", index=False)

## Image extraction

In [6]:
raw_panel = pd.read_csv(raw_panel_file)
raw_channel_order = pd.read_csv(raw_channel_order_file, names=["Metal Tag", "Channel"])
raw_panel_and_channel_order = pd.merge(raw_panel, raw_channel_order, on="Metal Tag")
source_channel_indices = raw_panel_and_channel_order.loc[
    raw_panel_and_channel_order["clean_target"].isin(source_channels), "Channel"
].tolist()
target_channel_indices = raw_panel_and_channel_order.loc[
    raw_panel_and_channel_order["clean_target"].isin(target_channels), "Channel"
].tolist()

source_img_dir = Path("source_img")
target_img_dir = Path("target_img")
source_img_dir.mkdir(exist_ok=True)
target_img_dir.mkdir(exist_ok=True)
for i, (source_img_file_name, target_img_file_name) in enumerate(layer_pairs):
    source_img = tifffile.imread(Path(raw_img_dir) / source_img_file_name)
    target_img = tifffile.imread(Path(raw_img_dir) / target_img_file_name)
    tifffile.imwrite(
        source_img_dir / f"{i:02d}_{source_img_file_name}",
        data=source_img[source_channel_indices],
    )
    tifffile.imwrite(
        target_img_dir / f"{i:02d}_{target_img_file_name}",
        data=target_img[target_channel_indices],
    )
    print(i, source_img_file_name, target_img_file_name)

0 28_9_ROI 10_10.tiff 27_8_ROI 09_9.tiff
1 54_1_ROI 01_1.tiff 55_2_ROI 02_2.tiff
2 79_6_ROI 06_6.tiff 80_7_ROI 07_7.tiff
3 106_12_ROI 13_13.tiff 107_13_ROI 14_14.tiff
4 129_2_ROI 02_2.tiff 130_3_ROI 03_3.tiff


## Image segmentation

In [7]:
app = Mesmer()
nuclear_source_channel_indices = [
    source_channels.index(channel) for channel in shared_nuclear_channels
]
nuclear_target_channel_indices = [
    target_channels.index(target) for target in shared_nuclear_channels
]

source_mask_dir = Path("source_masks")
source_mask_dir.mkdir(exist_ok=True)
for source_img_file in sorted(Path("source_img").glob("*.tiff")):
    source_img = tifffile.imread(source_img_file)
    nuclear_source_img = np.mean(source_img[nuclear_source_channel_indices], axis=0)
    source_mask = app.predict(
        np.expand_dims(np.stack((nuclear_source_img, nuclear_source_img), axis=-1), 0),
        compartment="nuclear",
    )[0, :, :, 0]
    tifffile.imwrite(source_mask_dir / source_img_file.name, data=source_mask)
    print(source_img_file.name)

target_mask_dir = Path("target_masks")
target_mask_dir.mkdir(exist_ok=True)
for target_img_file in sorted(Path("target_img").glob("*.tiff")):
    target_img = tifffile.imread(target_img_file)
    nuclear_target_img = np.mean(target_img[nuclear_target_channel_indices], axis=0)
    target_mask = app.predict(
        np.expand_dims(np.stack((nuclear_target_img, nuclear_target_img), axis=-1), 0),
        compartment="nuclear",
    )[0, :, :, 0]
    tifffile.imwrite(target_mask_dir / target_img_file.name, data=target_mask)
    print(target_img_file.name)

2022-03-04 16:14:32.847107: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-03-04 16:14:34.251876: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-04 16:14:34.251896: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IMLS-P-JWI): /proc/driver/nvidia/version does not exist
2022-03-04 16:14:34.252507: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2022-03-04 16:14:40.519199: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-04 16:14:40.536360: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2899885000 Hz


00_28_9_ROI 10_10.tiff
01_54_1_ROI 01_1.tiff
02_79_6_ROI 06_6.tiff
03_106_12_ROI 13_13.tiff
04_129_2_ROI 02_2.tiff
00_27_8_ROI 09_9.tiff
01_55_2_ROI 02_2.tiff
02_80_7_ROI 07_7.tiff
03_107_13_ROI 14_14.tiff
04_130_3_ROI 03_3.tiff


In [9]:
viewer = napari.Viewer()
image_layers = viewer.open(
    "target_img/00_27_8_ROI 09_9.tiff",
    layer_type="image",
    channel_axis=0,
    name=source_channels,
    colormap="gray",
    visible=False,
)
mask_layer = viewer.open("target_masks/00_27_8_ROI 09_9.tiff", layer_type="labels")